In [4]:
# In this script, we will calculate the strength of cloud feedbacks via the "adjustment" method
# By: Ty Janoski
# updated: 06.04.20

In [5]:
# import statements
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
import dask.array as da
import xesmf as xe

In [6]:
# read in lat/lons from albedo kernel for regridding
ds_ker = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/CAM5_kernels/alb.kernel.nc',use_cftime=True)
lats = ds_ker.lat
lons = ds_ker.lon

ds_out = xr.Dataset({'lat': (['lat'], lats),
                     'lon': (['lon'], lons),
                    }
                   )
del ds_ker

# we also need the adjusted TOA radiative perturbation from 4xCO2
nums = [str(n).zfill(2) for n in np.arange(1,13,1)]
diff = xr.concat([xr.open_dataset('/dx01/chiodo/data2/waccm/f1850.wcm.port.4xCO2.adj.001/diff.'+n+'.RAD.nc',
                                decode_times=False,drop_variables=['TCOOR','hyam','hybm','lev']) for n in nums],dim='time')
diff['time'] = np.arange(1,13,1)

# the WACCM output is not on the CAM5 grid, so it must be regridded
regridder = xe.Regridder(diff.FLNTC, ds_out, 'bilinear',reuse_weights=True,periodic=True)

# tile the files so that the dimensions match the abrupt4xCO2 CMIP5 data
G0_LW = -1 * da.tile(regridder(diff.FLNTC.transpose('time','lat','lon')),(150,1,1))
G_LW = -1 * da.tile(regridder(diff.FLNT.transpose('time','lat','lon')),(150,1,1))
G0_SW = da.tile(regridder(diff.FSNTC.transpose('time','lat','lon')),(150,1,1))
G_SW = da.tile(regridder(diff.FSNT.transpose('time','lat','lon')),(150,1,1))

Reuse existing file: bilinear_96x144_192x288_peri.nc


In [7]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
for mod in models:
    print('Reading in ' + mod + '...')
    
    # read in piControl data for climatology
    rsut = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rsut_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsut.isel(time=slice(-600,None))
    rsutcs = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rsutcs_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsutcs.isel(time=slice(-600,None))
    rlut = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rlut_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlut.isel(time=slice(-600,None))
    rlutcs = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rlutcs_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlutcs.isel(time=slice(-600,None))
    
    piControl = xr.merge([rsut,rsutcs,rlut,rlutcs])
    clim = piControl.groupby(piControl.time.dt.month).mean(dim='time')
    
    # read in abrupt4xCO2 data
    rsut = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rsut_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsut.isel(time=slice(None,1800))
    rsutcs = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rsutcs_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsutcs.isel(time=slice(None,1800))
    rlut = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rlut_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlut.isel(time=slice(None,1800))
    rlutcs = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rlutcs_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlutcs.isel(time=slice(None,1800))
    
    # regrid data to match CAM5 grid
    regridder = xe.Regridder(rsut, ds_out, 'bilinear',reuse_weights=True,periodic=True)
    with ProgressBar():
        dCRF_lw = regridder(((rlutcs - da.tile(clim.rlutcs,(150,1,1))) -
                             (rlut - da.tile(clim.rlut,(150,1,1)))).compute())
    with ProgressBar():
        dCRF_sw = regridder(((rsutcs - da.tile(clim.rsutcs,(150,1,1))) -
                             (rsut - da.tile(clim.rsut,(150,1,1)))).compute())

    # read in previously calculated clear-sky and all-sky feedbacks
    T = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_temp.nc',use_cftime=True)
    Q = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_wv.nc',use_cftime=True)
    alb = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_shell_albedo.nc',use_cftime=True)
    alb_regridder = xe.Regridder(alb,T,'bilinear',periodic=True,reuse_weights=True)
    alb = alb_regridder(alb)
    
    if(mod=='MIROC-ESM'):
        T['time'] = dCRF_lw.time
        Q['time'] = dCRF_lw.time
        alb['time'] = dCRF_lw.time
    
    # calculate radiative perturbation at TOA from clouds
    cloud_lw = dCRF_lw + (T.temp_LWCS - T.temp_LWAS) + (Q.wv_LWCS - Q.wv_LWAS) + (G0_LW - G_LW)
    cloud_sw = dCRF_sw + (Q.wv_SWCS - Q.wv_SWAS) + (alb.albedo_SWCS - alb.albedo_SWAS) + (G0_SW - G_SW)
    cloud_net = cloud_lw + cloud_sw
    
    print(cloud_net)
    cloud = xr.merge([cloud_lw.rename('cloud_LW'),cloud_sw.rename('cloud_SW'),cloud_net.rename('cloud_NET')])
    
    cloud.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_shell_cloud.nc')

Reading in ACCESS1-0...
Reuse existing file: bilinear_145x192_192x288_peri.nc
[########################################] | 100% Completed |  6.2s
[########################################] | 100% Completed |  6.2s
Reuse existing file: bilinear_64x128_192x288_peri.nc
using dimensions ('lat', 'lon') from data variable albedo_SWAS as the horizontal dimensions for this dataset.
<xarray.DataArray (time: 1800, lat: 192, lon: 288)>
dask.array<add, shape=(1800, 192, 288), dtype=float64, chunksize=(12, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 0300-01-16 12:00:00 ... 0449-12-16 12:00:00
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
Reading in ACCESS1-3...
Reuse existing file: bilinear_145x192_192x288_peri.nc
[########################################] | 100% Completed |  6.3s
[########################################] | 100% Completed |  5.9s
Reuse exi